<a href="https://colab.research.google.com/github/jdblackstar/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/121A_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

###***Part 1: Load and clean the data***

In [6]:
### YOUR CODE STARTS HERE

import numpy as np
import pandas as pd

# I previewed this file on my machine before uploading it, so I know that there
# are no column headers, so I'll open up the .names file and type those in here
column_headers = ['party', 'h-infants', 'water-proj', 'budget', 'phys-freeze', 'salvador-aid',
                  'rel-in-schools', 'anti-sat-test', 'nic-contras', 'mx-missile', 'immigration', 'synfuels-cut',
                  'edu-spend', 'sue-perfund', 'crime', 'd-free', 'export-sa']

df = pd.read_csv('house-votes-84.data')
print(df.shape)
df.head()

(434, 17)


,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [7]:
# Adding this column headers here
df = pd.read_csv('house-votes-84.data', names = column_headers)

print(df.shape)
df.head()

(435, 17)


,party,h-infants,water-proj,budget,phys-freeze,salvador-aid,rel-in-schools,anti-sat-test,nic-contras,mx-missile,immigration,synfuels-cut,edu-spend,sue-perfund,crime,d-free,export-sa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [8]:
# Want to deal with numeric values, not strings. Also replace '?' with NaN

df = df.replace({'y': 1, 'n': 0, '?': np.NaN})

df.head()

,party,h-infants,water-proj,budget,phys-freeze,salvador-aid,rel-in-schools,anti-sat-test,nic-contras,mx-missile,immigration,synfuels-cut,edu-spend,sue-perfund,crime,d-free,export-sa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [9]:
# Attempted to sort by party and display the whole table
# Was planning on seeing where GOP and Dem split in the graph and seperate from 
# there, but I found a better way ;D

from scipy import stats

df.sort_values(by=['party'])

df.head(435)

,party,h-infants,water-proj,budget,phys-freeze,salvador-aid,rel-in-schools,anti-sat-test,nic-contras,mx-missile,immigration,synfuels-cut,edu-spend,sue-perfund,crime,d-free,export-sa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
431,democrat,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
432,republican,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
433,republican,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0


In [10]:
# Going to call a describe here too, before I split, just in case I need it later

df.describe()

,h-infants,water-proj,budget,phys-freeze,salvador-aid,rel-in-schools,anti-sat-test,nic-contras,mx-missile,immigration,synfuels-cut,edu-spend,sue-perfund,crime,d-free,export-sa
count,423.000000,387.000000,424.000000,424.000000,420.000000,424.000000,421.000000,420.00000,413.000000,428.000000,414.000000,404.000000,410.000000,418.000000,407.000000,331.000000
mean,0.442080,0.503876,0.596698,0.417453,0.504762,0.641509,0.567696,0.57619,0.501211,0.504673,0.362319,0.423267,0.509756,0.593301,0.427518,0.812689
std,0.497222,0.500632,0.491140,0.493721,0.500574,0.480124,0.495985,0.49475,0.500605,0.500563,0.481252,0.494690,0.500516,0.491806,0.495327,0.390752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
# Make entirely new dataframes, splitting between GOP and Dem

gop = df[df['party'] == 'republican']
dem = df[df['party'] == 'democrat']

In [13]:
# Inspect gop dataframe to make sure it copied correctly
# as well as print a little more information about it

print(gop.head())
gop.describe()

         party  h-infants  water-proj  ...  crime  d-free  export-sa
0   republican        0.0         1.0  ...    1.0     0.0        1.0
1   republican        0.0         1.0  ...    1.0     0.0        NaN
7   republican        0.0         1.0  ...    1.0     NaN        1.0
8   republican        0.0         1.0  ...    1.0     0.0        1.0
10  republican        0.0         1.0  ...    1.0     0.0        0.0

[5 rows x 17 columns]


,h-infants,water-proj,budget,phys-freeze,salvador-aid,rel-in-schools,anti-sat-test,nic-contras,mx-missile,immigration,synfuels-cut,edu-spend,sue-perfund,crime,d-free,export-sa
count,165.000000,148.000000,164.000000,165.000000,165.000000,166.000000,162.000000,157.000000,165.000000,165.000000,159.000000,155.000000,158.000000,161.000000,156.000000,146.000000
mean,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534
std,0.391804,0.501652,0.341853,0.109760,0.215442,0.304104,0.428859,0.361010,0.320176,0.498186,0.339643,0.336322,0.347298,0.135649,0.286735,0.476168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
# Inspect dem dataframe to make sure it copied correctly
# as well as print a little more information about it

print(dem.head())
dem.describe()

      party  h-infants  water-proj  ...  crime  d-free  export-sa
2  democrat        NaN         1.0  ...    1.0     0.0        0.0
3  democrat        0.0         1.0  ...    0.0     0.0        1.0
4  democrat        1.0         1.0  ...    1.0     1.0        1.0
5  democrat        0.0         1.0  ...    1.0     1.0        1.0
6  democrat        0.0         1.0  ...    1.0     1.0        1.0

[5 rows x 17 columns]


,h-infants,water-proj,budget,phys-freeze,salvador-aid,rel-in-schools,anti-sat-test,nic-contras,mx-missile,immigration,synfuels-cut,edu-spend,sue-perfund,crime,d-free,export-sa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


###***Part 3: Find an issue that republicans support more than democrats with p<0.01***

In [15]:
# Begin running t-tests between gop and dem, using intuition to pick test
# starting points

stats.ttest_ind(gop['immigration'], dem['immigration'], nan_policy='omit')

Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)

In [19]:
# Republicans support immigration enforcement more than Democrats

print(gop['immigration'].mean())
print(dem['immigration'].mean())

0.5575757575757576
0.4714828897338403


In [21]:
stats.ttest_ind(gop['sue-perfund'], dem['sue-perfund'], nan_policy='omit')

# This satisfies the p-value requirement for the assignment, but let's see how 
# the votes break down

Ttest_indResult(statistic=13.51064251060933, pvalue=1.2278581709672758e-34)

In [22]:
print(gop['sue-perfund'].mean())
print(dem['sue-perfund'].mean())

# Republicans support this more than Democrats

0.8607594936708861
0.2896825396825397


###***Part 2: Find an issue that democrats support more than republicans with p<0.01***

In [23]:
stats.ttest_ind(gop['budget'], dem['budget'], nan_policy='omit')

# Another promising looking p-value, let's see the vote break down again

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

In [24]:
print(gop['budget'].mean())
print(dem['budget'].mean())

# Something that Democrats vote for more than Republicans

0.13414634146341464
0.8884615384615384


###***Part 4: Find an issue where the difference between republicans and democrats has p<0.1***

In [25]:
# Testing satellite testing

stats.ttest_ind(gop['anti-sat-test'], dem['anti-sat-test'], nan_policy='omit')

# Not a promising p-value

Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)

In [28]:
# Testing the duty free data

stats.ttest_ind(gop['d-free'], dem['d-free'], nan_policy='omit')

# Another non-promising p-value

Ttest_indResult(statistic=-12.853146132542978, pvalue=5.997697174347365e-32)

In [29]:
stats.ttest_ind(gop['water-proj'], dem['water-proj'], nan_policy='omit')

# This is the closest we've gotten, let's look at the breakdown to see
# how the difference in votes is expressed

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

In [30]:
print(gop['water-proj'].mean())
print(dem['water-proj'].mean())

# Pretty close to 50:50

0.5067567567567568
0.502092050209205


## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!